In [2]:
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import genfromtxt
import pandas as pd
import plotly.express as px
pd.options.plotting.backend = "plotly"
import matplotlib.pyplot as plt
import time

import tensorflow as tf
from tensorflow.keras.metrics import BinaryAccuracy,FalsePositives,FalseNegatives,TruePositives,TrueNegatives,Precision,Recall,AUC
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau
 
import sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,roc_curve
from sklearn import preprocessing


In [3]:

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


In [4]:
## LOAD TRAIN DATA
X = genfromtxt('C:/Users/NSF/Desktop/boyang work/final/XMEGA/DATASETS/ethan_mabon_capture/labels/traces_train_1000_20_2.csv', delimiter=',')
Y = genfromtxt('C:/Users/NSF/Desktop/boyang work/final/XMEGA/DATASETS/ethan_mabon_capture/labels/label_train_1000_20_2.csv', delimiter=',')
Y=np.asarray(Y).astype('float32')
print("Shape: ",np.shape(X))
print("Shape: ",np.shape(Y))

Shape:  (75842, 1000)
Shape:  (75842,)


In [5]:
#Ditrubution of Data in Training data
pd.value_counts(np.array(Y))

0.0    74851
1.0      991
dtype: int64

In [6]:
# NORMALIZING Train data
X[np.isinf(X)] = 0
Y[np.isinf(Y)] = 0
for i in range(np.shape(X)[0]):
    X[i]=preprocessing.normalize([X[i]])

In [7]:
# Spliting Train data with 10% for validation.
X_train, X_val, y_train, y_val = train_test_split(X,Y,test_size=0.1,random_state=42)

print("Shape/ Traces TRAIN: ",np.shape(X_train))
print("Shape/ Label TRAIN",np.shape(y_train))
print("Shape/ Traces VAL: ",np.shape(X_val))
print("Shape/ Label VAL",np.shape(y_val))

Shape/ Traces TRAIN:  (68257, 1000)
Shape/ Label TRAIN (68257,)
Shape/ Traces VAL:  (7585, 1000)
Shape/ Label VAL (7585,)


In [8]:
y_train=tf.keras.utils.to_categorical(
    y_train, num_classes=2, dtype='float32'
)
y_val=tf.keras.utils.to_categorical(
    y_val, num_classes=2, dtype='float32'
)

In [9]:
#Loading Test Data
X_test = genfromtxt('C:/Users/NSF/Desktop/boyang work/final/XMEGA/DATASETS/ethan_mabon_capture/labels/traces_test_1000_20_20.csv', delimiter=',')
Y = genfromtxt('C:/Users/NSF/Desktop/boyang work/final/XMEGA/DATASETS/ethan_mabon_capture/labels/label_test_1000_20_20.csv', delimiter=',')

In [10]:
# NORMALIZING Test data
X_test[np.isinf(X_test)] = 0
Y[np.isinf(Y)] = 0
for i in range(np.shape(X_test)[0]):
    X_test[i]=preprocessing.normalize([X_test[i]])
y_test=np.asarray(Y).astype('float32')
y_test=tf.keras.utils.to_categorical(
    y_test, num_classes=2, dtype='float32'
)

In [11]:
# Test Data Structure:
print("Shape/ Traces TEST: ",np.shape(X_test))
print("Shape/ Label TEST",np.shape(y_test))

Shape/ Traces TEST:  (237451, 1000)
Shape/ Label TEST (237451, 2)


In [12]:
#Dynamic Model Definition

model=Sequential()
model.add(Dense(np.shape(X)[1],input_dim=len(X[0,:]),activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='softmax'))
#model.add(Dense(2,activation='sigmoid'))


In [13]:
#Compile Model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 128)               128128    
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,141,674
Trainable params: 1,141,674
Non-trainable params: 0
_________________________________________________________________


In [14]:
#Model Metrics
metrics=[TruePositives(name='tp'),TrueNegatives(name='tn'),FalsePositives(name='fp'),FalseNegatives(name='fn'),BinaryAccuracy(name='Binary_Accuracy'),Precision(name="Precision"),Recall(name='Recall'),AUC(name='AUC')]

In [15]:
save_file_name="best_model.hdf5"

In [16]:
#Training checkpoints
checkpointer = ModelCheckpoint(save_file_name,
                               monitor='Binary_Accuracy',
                               verbose=1,
                               # save_weights_only=True,
                               # save_best_only=True,
                               mode='max')

In [17]:
#Dynamic Learing Rate on a Platue
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=3e-6)

In [18]:
#Callbacks
callbacks = [checkpointer,reduce_lr]

In [19]:
opt = keras.optimizers.Adam()
model.compile(loss='BinaryCrossentropy',optimizer=opt,metrics=metrics)

In [ ]:

history = model.fit(
    X_train,
    y_train,
    batch_size=10,
    epochs=140,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_val, y_val),
    shuffle=True,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/140
6819/6826 [============================>.] - ETA: 0s - loss: 0.0789 - tp: 67301.0000 - tn: 67301.0000 - fp: 889.0000 - fn: 889.0000 - Binary_Accuracy: 0.9870 - Precision: 0.9870 - Recall: 0.9870 - AUC: 0.9870
Epoch 1: saving model to best_model.hdf5
6826/6826 [==============================] - 32s 4ms/step - loss: 0.0789 - tp: 67368.0000 - tn: 67368.0000 - fp: 889.0000 - fn: 889.0000 - Binary_Accuracy: 0.9870 - Precision: 0.9870 - Recall: 0.9870 - AUC: 0.9870 - val_loss: 0.0817 - val_tp: 7476.0000 - val_tn: 7476.0000 - val_fp: 109.0000 - val_fn: 109.0000 - val_Binary_Accuracy: 0.9856 - val_Precision: 0.9856 - val_Recall: 0.9856 - val_AUC: 0.9856 - lr: 0.0010
Epoch 2/140
6818/6826 [============================>.] - ETA: 0s - loss: 0.0715 - tp: 67299.0000 - tn: 67299.0000 - fp: 881.0000 - fn: 881.0000 - Binary_Accuracy: 0.9871 - Precision: 0.9871 - Recall: 0.9871 - AUC: 0.9871
Epoch 2: saving model to best_model.hdf5
6826/6826 [==============================] - 30s 4ms/step -

6812/6826 [============================>.] - ETA: 0s - loss: 0.0059 - tp: 67991.0000 - tn: 67991.0000 - fp: 129.0000 - fn: 129.0000 - Binary_Accuracy: 0.9981 - Precision: 0.9981 - Recall: 0.9981 - AUC: 0.9999
Epoch 13: saving model to best_model.hdf5
6826/6826 [==============================] - 26s 4ms/step - loss: 0.0059 - tp: 68128.0000 - tn: 68128.0000 - fp: 129.0000 - fn: 129.0000 - Binary_Accuracy: 0.9981 - Precision: 0.9981 - Recall: 0.9981 - AUC: 0.9999 - val_loss: 0.3133 - val_tp: 7442.0000 - val_tn: 7442.0000 - val_fp: 143.0000 - val_fn: 143.0000 - val_Binary_Accuracy: 0.9811 - val_Precision: 0.9811 - val_Recall: 0.9811 - val_AUC: 0.9846 - lr: 4.0000e-05
Epoch 14/140
6816/6826 [============================>.] - ETA: 0s - loss: 0.0032 - tp: 68089.0000 - tn: 68089.0000 - fp: 71.0000 - fn: 71.0000 - Binary_Accuracy: 0.9990 - Precision: 0.9990 - Recall: 0.9990 - AUC: 1.0000
Epoch 14: saving model to best_model.hdf5
6826/6826 [==============================] - 26s 4ms/step - loss: 

Epoch 25/140
6819/6826 [============================>.] - ETA: 0s - loss: 5.8393e-05 - tp: 68189.0000 - tn: 68189.0000 - fp: 1.0000 - fn: 1.0000 - Binary_Accuracy: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - AUC: 1.0000
Epoch 25: saving model to best_model.hdf5
6826/6826 [==============================] - 26s 4ms/step - loss: 5.8340e-05 - tp: 68256.0000 - tn: 68256.0000 - fp: 1.0000 - fn: 1.0000 - Binary_Accuracy: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - AUC: 1.0000 - val_loss: 0.6730 - val_tp: 7439.0000 - val_tn: 7439.0000 - val_fp: 146.0000 - val_fn: 146.0000 - val_Binary_Accuracy: 0.9808 - val_Precision: 0.9808 - val_Recall: 0.9808 - val_AUC: 0.9824 - lr: 3.0000e-06
Epoch 26/140
6825/6826 [============================>.] - ETA: 0s - loss: 5.2226e-05 - tp: 68249.0000 - tn: 68249.0000 - fp: 1.0000 - fn: 1.0000 - Binary_Accuracy: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - AUC: 1.0000
Epoch 26: saving model to best_model.hdf5
6826/6826 [==============================] - 26s 4m

Epoch 37/140
4627/6826 [===================>..........] - ETA: 7s - loss: 2.0207e-05 - tp: 46270.0000 - tn: 46270.0000 - fp: 0.0000e+00 - fn: 0.0000e+00 - Binary_Accuracy: 1.0000 - Precision: 1.0000 - Recall: 1.0000 - AUC: 1.0000

In [ ]:
# save_path = f'model/Custom_{int(time.time())}_model'
# print(save_path)
# model.save(save_path)

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=20)
print("test loss, test acc:", results)


In [ ]:
# summarize history for accuracy
print(history.history.keys())

plt.plot(history.history['Binary_Accuracy'])
plt.plot(history.history['val_Binary_Accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict(X_test)
pred1=pred
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)

In [ ]:
thr=9.99999821
#CM = confusion_matrix(y_true, pred>thr)
CM = confusion_matrix(y_true, pred)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
plt.show()


In [ ]:
tn, fp, fn, tp = CM.ravel()
def accuracy(tn, fp, fn, tp):
    return (tp + tn) / (tp + tn + fp + fn)

In [ ]:
def precision(tn, fp, fn, tp):
    return(tp/(tp+fp))

In [ ]:
def recall(tn, fp, fn, tp):
    return(tp/(tp+fn))

In [ ]:
"Accuracy: {0}".format(accuracy(tn, fp, fn, tp))

In [ ]:
"Precision: {0}".format(precision(tn, fp, fn, tp))

In [ ]:
"Recall: {0}".format(recall(tn, fp, fn, tp))

In [ ]:
labels=np.array([i[0] for i in y_test])

In [ ]:
fp,tp,thresholds=roc_curve(labels,pred1[:,0])

In [ ]:
plt.plot(fp,tp)
plt.title("CLASS 0")
plt.grid()

skip=5
for i in range(0,len(thresholds),skip):
    plt.text(fp[i],tp[i],thresholds[i])

In [ ]:
fp2,tp2,thresholds2=roc_curve(labels,pred1[:,1])

In [ ]:
plt.plot(fp2,tp2)
plt.title("CLASS 1")
plt.grid()

skip=5
for i in range(0,len(thresholds2),skip):
    plt.text(fp2[i],tp2[i],thresholds2[i])

In [ ]:
X = genfromtxt('data/test/Trace2/traces_test_1000_1_1.csv', delimiter=',')


In [ ]:
for i in range(np.shape(X)[0]):
    X[i]=preprocessing.normalize([X[i]])


In [ ]:
pred = model.predict(X)
pred1=pred
pred = np.argmax(pred,axis = 1) 

In [ ]:
savetxt('predictions/Train_trace1_1000_20_2_Test_trace2_1000_1.csv', pred, delimiter=',')